In [1]:
#LIBRARIES 
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
import pandas as pd
from datetime import datetime, timedelta

In [2]:
from trino.dbapi import connect
from trino.auth import BasicAuthentication

In [3]:
conn = connect(host="tcp.smooth-bonefish.dataos.app",
      port="7432",
      auth=BasicAuthentication('balaji', 'dG9rZW5fc2Vjb25kbHlfZWFybHlfc2hhcnBfZmxlYS43YTU4NDNkOC02MzE4LTQ3MzQtYjZjOS05NTZlOWNlNGI1NWM='),
      http_scheme="https",
      http_headers={"cluster-name": "minervaa"}  # eg:minervaa
      )


In [19]:
train_df = pd.read_csv("data/training_data.csv")

In [20]:
train_df

,profile_ID,DOB,Contact_type,Gender,Pack_ID,Start_Date,Status,Upgradable,Validity_End,Country,State,avg_dur,num_devs,freq_used_dev,Subscription Type,Billing Type,Customer Longevity
0,rsanchez,6/10/2005,Email,Female,432281,10/28/2021,INACTIVE,YES,12/5/2022,India,Goa,1512.277778,18,Android Phone,12,Paytm,4
1,karina10,1/12/2005,Email,Female,432300,2/10/2021,INACTIVE,NO,11/22/2022,India,Goa,1329.500000,8,iPad,3,NetBanking,2
2,sethconrad,10/30/1992,Email,Male,432410,10/30/2022,ACTIVE,YES,11/10/2022,India,Goa,1478.166667,18,PC/Laptop,6,NetBanking,3
3,ewilson,8/14/2015,Email,Female,432496,1/14/2022,ACTIVE,NO,12/27/2022,India,Goa,1118.133333,15,PC/Laptop,12,Paytm,4
4,evanvelasquez,5/25/1992,Email,Female,432553,1/21/2022,INACTIVE,YES,11/9/2022,India,Goa,1557.428571,21,FireTV,12,Paytm,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,brandili,12/14/2021,WhatsApp,Female,432660,3/22/2022,INACTIVE,NO,12/26/2022,India,Arunachal Pradesh,1266.307692,13,PC/Laptop,12,Paytm,4
996,davidlowe,4/1/1993,WhatsApp,Male,432693,9/25/2021,INACTIVE,YES,11/14/2022,India,Arunachal Pradesh,1199.000000,18,FireTV,12,Paytm,4
997,carrierivera,9/30/2002,WhatsApp,Female,432771,11/24/2021,INACTIVE,YES,12/3/2022,India,Arunachal Pradesh,1482.000000,17,FireTV,12,Paytm,4
998,ronniecoffey,8/29/2002,WhatsApp,Female,432802,1/21/2022,ACTIVE,NO,12/16/2022,India,Arunachal Pradesh,1518.000000,30,PC/Laptop,12,Paytm,4


In [4]:
player_qr = "SELECT * FROM icebase.telco_ott.ott_player where date_diff('day', session_start, current_date) <= 1"


In [5]:
player_df = pd.read_sql(player_qr, conn)

/Users/astrobalaji/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [6]:
player_df

,__metadata,event_type,session_start,timezone,subscribed,platform,partner_id,application_version,location,hardware,...,delivery_protocol,playing_rate,starting_bitrate,video_resolution,start_position,end_position,user_id,device_id,video_position,duration
0,{'_dataos_run_mapper_id': '299f3d53-d7c6-48ca-...,stop,2022-12-27 08:20:28.000 UTC,Asia/Amman,True,iOS,sonyliv,6.4,Palwal,Apple TV,...,Cloudfront,,7.8,480p,00:01,02:14,knguyen,62jqy-8754f-rq29u-Yt9yg,01:21,4820.0
1,{'_dataos_run_mapper_id': '299f3d53-d7c6-48ca-...,play,2022-12-27 08:20:06.000 UTC,Asia/Bishkek,True,iOS,sonyliv,6.7,Dimapur,iPad,...,Akamai,,5.6,480p,00:19,00:25,dianamerritt,38NGE-4452Q-Ez03a-uO9ye,00:24,272.0
2,{'_dataos_run_mapper_id': '299f3d53-d7c6-48ca-...,skip,2022-12-27 08:30:30.000 UTC,Asia/Colombo,False,iOS,hotstar,2.2,Jalgaon,iPad,...,Akamai,,6.9,720p,00:02,02:57,jason99,82EvW-9392F-mi04P-wF4Vs,01:43,6031.0
3,{'_dataos_run_mapper_id': '299f3d53-d7c6-48ca-...,stop,2022-12-27 07:50:53.000 UTC,America/St_Kitts,False,iOS,sonyliv,5.0,Panvel,iPad,...,Cloudfront,,7.5,720p,00:11,00:33,ssmith,15rnh-4118H-Vs59o-dB6Us,00:32,1266.0
4,{'_dataos_run_mapper_id': '299f3d53-d7c6-48ca-...,skip,2022-12-27 08:00:30.000 UTC,America/Havana,True,iOS,sonyliv,1.4,Ballia,iPhone,...,Cloudfront,,7.1,4K,00:05,03:06,terrellchristy,31BYf-3847g-bZ94a-Kg4Im,03:06,10865.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2000,{'_dataos_run_mapper_id': 'ad3fafa4-f726-4df2-...,stop,2022-12-27 08:40:40.000 UTC,Africa/Mbabane,False,iOS,sonyliv,4.8,Kadapa[18],iPad,...,Akamai,,8.5,720p,00:03,00:48,jessicacobb,48cpS-9821f-yu18I-rR2BJ,00:15,724.0
2001,{'_dataos_run_mapper_id': 'ad3fafa4-f726-4df2-...,stop,2022-12-27 08:40:52.000 UTC,Asia/Thimphu,False,iOS,sonyliv,1.6,Bijapur,Apple TV,...,Cloudfront,,7.9,1080p,00:28,01:00,mariabolton,76rMP-9220q-rc09z-GS6og,00:34,376.0
2002,{'_dataos_run_mapper_id': 'ad3fafa4-f726-4df2-...,skip,2022-12-27 08:40:13.000 UTC,America/Managua,False,iOS,hotstar,4.8,Nagaon,iPhone,...,Cloudfront,,6.0,480p,00:20,00:37,ericreed,70iTi-3726L-Cy76Y-xx2va,00:24,242.0
2003,{'_dataos_run_mapper_id': 'ad3fafa4-f726-4df2-...,play,2022-12-27 08:40:05.000 UTC,Asia/Bahrain,False,Android,hotstar,2.4,Cuttack,Android TV,...,Cloudfront,,5.1,720p,00:06,01:20,alexandriareid,62CeD-2384S-XX99Z-aU1fl,00:22,998.0


In [7]:
userids = list(player_df['user_id'].unique())

In [8]:
userids = ["'"+u+"'" for u in userids]

In [23]:
user_qr = """SELECT profile_id as user_id, dob, contact_type, gender, pack_id,
start_date, status, upgradable, validity_end, country, state, customer_id
FROM icebase.telco_ott.ott_userdataset
WHERE profile_id IN ({0})
""".format(",".join(userids))

In [24]:
user_df = pd.read_sql(user_qr, conn)

In [17]:
billing_qr = """SELECT  customer_id, billing_type, customer_longevity,subscription_type
FROM icebase.telco_ott.ott_billing_01
WHERE billing_type IN ({0})""".format(",".join(userids))

In [18]:
billing_df = pd.read_sql(billing_qr, conn)

In [27]:
userids

["'knguyen'",
 "'dianamerritt'",
 "'jason99'",
 "'ssmith'",
 "'terrellchristy'",
 "'rosenicholas'",
 "'michael32'",
 "'ugraham'",
 "'michael87'",
 "'maddoxjustin'",
 "'brandycruz'",
 "'rcompton'",
 "'smithgabrielle'",
 "'zhuerta'",
 "'mjenkins'",
 "'adamsamy'",
 "'wrightjohn'",
 "'vsims'",
 "'spencerjeremy'",
 "'bensonsusan'",
 "'samantha08'",
 "'perezsuzanne'",
 "'aharris'",
 "'dave07'",
 "'williamsjeremy'",
 "'diana76'",
 "'robert24'",
 "'michaelharris'",
 "'pottserica'",
 "'paula88'",
 "'christophercrane'",
 "'evansthomas'",
 "'fpalmer'",
 "'davisjay'",
 "'lmanning'",
 "'ulewis'",
 "'rubenmendoza'",
 "'barrerakelly'",
 "'jacobdurham'",
 "'kelly59'",
 "'hardingandrea'",
 "'jenniferstark'",
 "'tamara26'",
 "'bryansteele'",
 "'jhoffman'",
 "'scottnelson'",
 "'michelle49'",
 "'michellebean'",
 "'rbender'",
 "'carol47'",
 "'melissaferguson'",
 "'dennisjeanne'",
 "'oschneider'",
 "'jason07'",
 "'noahwhite'",
 "'evansrobert'",
 "'williamsalejandro'",
 "'darrelllindsey'",
 "'fgonzalez'",
 "

In [31]:
user_qr = "SELECT * FROM icebase.telco_ott.ott_userdataset"

In [32]:
user_df = pd.read_sql(user_qr, conn)

In [25]:
user_df

,user_id,dob,contact_type,gender,pack_id,start_date,status,upgradable,validity_end,country,state,customer_id
0,michael87,2021-12-30 00:00:00.000 UTC,Telegram,Male,432898,2021-09-26 00:00:00.000 UTC,ACTIVE,NO,2022-11-27 00:00:00.000 UTC,India,Rajasthan,8773-HHUOZ
1,fgonzalez,1992-02-09 00:00:00.000 UTC,WhatsApp,Male,432352,2021-01-13 00:00:00.000 UTC,ACTIVE,YES,2022-12-22 00:00:00.000 UTC,India,Andhra Pradesh,9574-BOSMD
2,jennifer81,1992-10-03 00:00:00.000 UTC,Email,Male,432847,2022-03-24 00:00:00.000 UTC,INACTIVE,YES,2022-12-22 00:00:00.000 UTC,India,Sikkim,8035-BUYVG


In [38]:
billing_df

,__metadata,customer_id,billing_id,subscription_type_,billing_type_,customer_longevity
0,{'_dataos_run_mapper_id': 'ae577baa-306e-400f-...,1719964,432166,1 Months,Paytm,1 Years
1,{'_dataos_run_mapper_id': 'ae577baa-306e-400f-...,1231297,432168,1 Months,Paytm,1 Years
2,{'_dataos_run_mapper_id': 'ae577baa-306e-400f-...,1847764,432180,1 Months,Paytm,1 Years
3,{'_dataos_run_mapper_id': 'ae577baa-306e-400f-...,1888210,432190,1 Months,Paytm,1 Years
4,{'_dataos_run_mapper_id': 'ae577baa-306e-400f-...,1188917,432197,1 Months,Paytm,1 Years
...,...,...,...,...,...,...
995,{'_dataos_run_mapper_id': 'ae577baa-306e-400f-...,1782237,433131,6 Months,NetBanking,3 Years
996,{'_dataos_run_mapper_id': 'ae577baa-306e-400f-...,1755205,433132,6 Months,NetBanking,3 Years
997,{'_dataos_run_mapper_id': 'ae577baa-306e-400f-...,1490831,433141,6 Months,NetBanking,3 Years
998,{'_dataos_run_mapper_id': 'ae577baa-306e-400f-...,1609685,433149,6 Months,NetBanking,3 Years


In [33]:
user_df

,__metadata,profile_id,contact_type,dob,email,first_name,gender,mobile_number,pack_id,start_date,status,upgradable,validity_end,country,state,customer_id,zipcode
0,{'_dataos_run_mapper_id': 'b0c4117c-95c8-48ee-...,ewilson,Email,2015-08-14 00:00:00.000 UTC,elavalde9g@google.ru,Eugenia,Female,711-627-9420,432496,2022-01-14 00:00:00.000 UTC,ACTIVE,NO,2022-12-27 00:00:00.000 UTC,India,Goa,0289-IVARM,None
1,{'_dataos_run_mapper_id': 'b0c4117c-95c8-48ee-...,xbrooks,Email,2021-12-16 00:00:00.000 UTC,bgareof@mlb.com,Brennan,Male,879-467-5949,433035,2021-08-05 00:00:00.000 UTC,ACTIVE,NO,2022-11-12 00:00:00.000 UTC,India,Goa,9306-CPCBC,30935 CEDEX 9
2,{'_dataos_run_mapper_id': 'b0c4117c-95c8-48ee-...,gregorycassandra,Email,1992-05-30 00:00:00.000 UTC,grickellc9@dailymotion.com,Garreth,Male,462-832-7210,432597,2022-05-11 00:00:00.000 UTC,ACTIVE,NO,2022-11-13 00:00:00.000 UTC,India,Assam,9514-JDSKI,959-1287
3,{'_dataos_run_mapper_id': 'b0c4117c-95c8-48ee-...,robin12,Email,1992-10-25 00:00:00.000 UTC,jkybbyei2@icio.us,Janith,Female,635-992-1494,432806,2021-10-12 00:00:00.000 UTC,ACTIVE,NO,2022-12-12 00:00:00.000 UTC,India,Assam,8499-BRXTD,33-321
4,{'_dataos_run_mapper_id': 'b0c4117c-95c8-48ee-...,aguilarkim,Email,2005-12-10 00:00:00.000 UTC,audale3t@livejournal.com,Anthia,Female,865-392-1991,432293,2021-07-16 00:00:00.000 UTC,ACTIVE,NO,2022-12-22 00:00:00.000 UTC,India,Bihar,5940-NFXKV,6720
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,{'_dataos_run_mapper_id': 'b0c4117c-95c8-48ee-...,berrycarla,WhatsApp,1993-12-02 00:00:00.000 UTC,okippingra@mozilla.com,Olivia,Female,708-330-9061,433138,2021-07-08 00:00:00.000 UTC,INACTIVE,YES,2022-12-19 00:00:00.000 UTC,India,Madhya Pradesh,3873-WOSBC,None
996,{'_dataos_run_mapper_id': 'b0c4117c-95c8-48ee-...,fergusonnancy,WhatsApp,2002-10-09 00:00:00.000 UTC,bgatheralhe@flavors.me,Barney,Male,786-408-7194,432782,2021-04-29 00:00:00.000 UTC,INACTIVE,YES,2022-12-11 00:00:00.000 UTC,India,Himachal Pradesh,3009-JWMPU,None
997,{'_dataos_run_mapper_id': 'b0c4117c-95c8-48ee-...,sullivanerika,WhatsApp,1992-07-19 00:00:00.000 UTC,dzammitt4t@opera.com,Dayna,Female,718-510-1258,432329,2021-03-08 00:00:00.000 UTC,INACTIVE,YES,2022-11-20 00:00:00.000 UTC,India,Arunachal Pradesh,8963-JLGJT,49-345
998,{'_dataos_run_mapper_id': 'b0c4117c-95c8-48ee-...,davidlowe,WhatsApp,1993-01-04 00:00:00.000 UTC,ctothex@google.com.hk,Chic,Male,472-131-0900,432693,2021-09-25 00:00:00.000 UTC,INACTIVE,YES,2022-11-14 00:00:00.000 UTC,India,Arunachal Pradesh,3466-BYAVD,None


In [34]:
b2b_qr="SELECT*FROM icebase.telco_ott.ott_b2bdata"

In [35]:
complaints_qr="SELECT*FROM icebase.telco_ott.ott_complaints"

In [36]:
userquery = """SELECT platform, cdn 
FROM icebase.telco_ott.ott_complaints
WHERE cdn IN ({0})""".format(",".join(userids))

In [37]:
complaints_df = pd.read_sql(userquery, conn)

In [38]:
complaints_df

,platform,cdn


In [40]:
contract_qr="SELECT*FROM icebase.telco_ott.ott_contract"

In [41]:
userquery = """SELECT  Contract Term, Contract Amount 
FROM icebase.telco_ott.ott_contract
WHERE Contract Term IN ({0})""".format(",".join(userids))

In [46]:
##contract_df = pd.read_sql(userquery, conn)